# Which factories are responsible for which chemical releases? Carefully describe how you determined this using all the data you have available. For the factories you identified, describe any observed patterns of operation revealed in the data.

In [1]:
# import libraries
import numpy as np
import pandas as pd
from pandasql import sqldf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pprint
from tabulate import tabulate
import operator
sns.set_style("darkgrid")

In [2]:
# import data
sensor = pd.read_csv('https://raw.githubusercontent.com/j-tice/capstone_final/master/data/prepared_data/clean_data/clean_sensor_data.csv')
weather = pd.read_csv('https://raw.githubusercontent.com/j-tice/capstone_final/ef9ac7bceb5cde6fd932cb5c0018ca90329a9d3c/data/prepared_data/clean_data/clean_meteorological_data.csv')
locations = pd.read_csv('https://raw.githubusercontent.com/j-tice/capstone_final/ef9ac7bceb5cde6fd932cb5c0018ca90329a9d3c/data/prepared_data/clean_data/locations.csv')
chem_counts = pd.read_csv('https://raw.githubusercontent.com/j-tice/capstone_final/a9b08f3c0f6fdd6e72bdd1465f3c6177b687d332/data/prepared_data/clean_data/chem_counts.csv')

psql = lambda q: sqldf(q, globals())

In [3]:
# eliminate the potentially faulty sensor 8
sensor = sensor.loc[sensor.Monitor != 8]

In [4]:
# determine cutoffs for each chemical type
chems = list(sensor.Chemical.unique())
chem_dfs = dict()
cutoff_values = dict()

for chemical in chems:
    chem_dfs['{name}'.format(name=chemical)] = sensor.loc[sensor.Chemical == chemical]
    
for chemical in chems:
    reference = chem_dfs[chemical]
    tmp_cutoff = np.mean(reference.Reading) + (3 * np.std(reference.Reading))
    cutoff_values['{name}'.format(name=chemical)] = round(tmp_cutoff, 4)

pprint.pprint(cutoff_values)

{'AGOC-3A': 10.6913,
 'Appluimonia': 2.699,
 'Chlorodinine': 3.3325,
 'Methylosmolene': 8.7986}


In [5]:
# isolate observations where the chemical reading is above the cutoff

spikes = dict()
for chemical in chems:
    tmp_df = chem_dfs[chemical]
    tmp_cutoff = cutoff_values[chemical]
    tmp_outliers = tmp_df.loc[tmp_df.Reading > tmp_cutoff]
    spikes['{name}_outliers'.format(name=chemical)] = tmp_outliers

In [6]:
# lets see the count of each significant chemical by sensor
for key in spikes:   
    sigs = spikes[key][['Monitor', 'Chemical', 'Reading']] \
        .groupby(by=['Chemical', 'Monitor']).count().sort_values(by='Reading', ascending=False)
    print(tabulate(sigs, headers=['Chemical, Sensor', 'Number of Significant Readings'], tablefmt='psql'))
    print()    

+-----------------------+----------------------------------+
| Chemical, Sensor      |   Number of Significant Readings |
|-----------------------+----------------------------------|
| ('Methylosmolene', 6) |                               26 |
| ('Methylosmolene', 3) |                               16 |
| ('Methylosmolene', 4) |                               10 |
| ('Methylosmolene', 5) |                               10 |
| ('Methylosmolene', 7) |                               10 |
| ('Methylosmolene', 9) |                                5 |
| ('Methylosmolene', 2) |                                3 |
| ('Methylosmolene', 1) |                                1 |
+-----------------------+----------------------------------+

+---------------------+----------------------------------+
| Chemical, Sensor    |   Number of Significant Readings |
|---------------------+----------------------------------|
| ('Chlorodinine', 3) |                              111 |
| ('Chlorodinine', 6) |        

In [7]:
# let's find some of the instances that drive the above charts, specific perpetraitors, and track origin with wind data

In [8]:
top_10s = dict()
for key in spikes:
    top_10s['top_{name}'.format(name=key)] = spikes[key].sort_values(by='Reading', ascending=False).head(10)

In [9]:
for elem in top_10s:
    print('+---' + elem.upper() + ':\n')
    pprint.pprint(top_10s[elem])
    print('\n')

+---TOP_METHYLOSMOLENE_OUTLIERS:

             Chemical  Monitor            Date_Time    Reading
59299  Methylosmolene        6  2016-12-08 22:00:00  100.77640
6876   Methylosmolene        6  2016-04-09 01:00:00   94.34631
972    Methylosmolene        6  2016-04-02 04:00:00   88.53254
53498  Methylosmolene        6  2016-12-02 04:00:00   84.95453
53462  Methylosmolene        6  2016-12-02 03:00:00   80.54764
53534  Methylosmolene        6  2016-12-02 05:00:00   79.04791
26676  Methylosmolene        3  2016-08-01 23:00:00   75.96199
61171  Methylosmolene        6  2016-12-11 02:00:00   68.88227
56090  Methylosmolene        7  2016-12-05 04:00:00   63.31599
26822  Methylosmolene        2  2016-08-02 04:00:00   58.46353


+---TOP_CHLORODININE_OUTLIERS:

           Chemical  Monitor            Date_Time   Reading
26894  Chlorodinine        2  2016-08-02 06:00:00  15.72311
71647  Chlorodinine        6  2016-12-23 05:00:00  15.04032
67436  Chlorodinine        4  2016-12-18 08:00:00  14.58982

In [10]:
# now lets join in the sensor coords and weather data to that specific timestamp
# ...and do some weird unpacking using the dicts __getitem__ method so we can use some sql joins

top_app = operator.itemgetter('top_Appluimonia_outliers')(top_10s)
top_agoc = operator.itemgetter('top_AGOC-3A_outliers')(top_10s)
top_chloro = operator.itemgetter('top_Chlorodinine_outliers')(top_10s)
top_methyl = operator.itemgetter('top_Methylosmolene_outliers')(top_10s)


In [11]:
clean_locs = locations[locations.Name.str.contains('Sensor')]
clean_locs.columns = 'monitor,x,y'.split(',')
clean_locs.monitor = list(range(1,10))
clean_locs = clean_locs.reset_index().drop(['index'], axis='columns')

table_names = ['top_app', 'top_agoc', 'top_chloro', 'top_methyl']

q_app = '''
WITH base AS (SELECT Chemical, Monitor, Date_Time, Reading FROM {table} ),
base_locations AS (SELECT * FROM base LEFT JOIN clean_locs ON base.Monitor = clean_locs.monitor ),
result AS (SELECT * FROM base_locations LEFT JOIN weather ON base_locations.Date_Time = weather.date )
SELECT 
  Chemical  AS chemical, 
  Monitor   AS monitor, 
  Date_time AS reading_stamp,
  Reading   AS reading,
  x         AS sensor_x, 
  y         AS sensor_y,
  date      AS weather_stamp, 
  wind_direction, wind_speed
FROM result WHERE date IS NOT NULL'''.format(table=table_names[0])

q_agoc = '''
WITH base AS (SELECT Chemical, Monitor, Date_Time, Reading FROM {table} ),
base_locations AS (SELECT * FROM base LEFT JOIN clean_locs ON base.Monitor = clean_locs.monitor ),
result AS (SELECT * FROM base_locations LEFT JOIN weather ON base_locations.Date_Time = weather.date )
SELECT 
  Chemical  AS chemical, 
  Monitor   AS monitor, 
  Date_time AS reading_stamp,
  Reading   AS reading,
  x         AS sensor_x, 
  y         AS sensor_y,
  date      AS weather_stamp, 
  wind_direction, wind_speed
FROM result WHERE date IS NOT NULL'''.format(table=table_names[1])

q_chloro = '''
WITH base AS (SELECT Chemical, Monitor, Date_Time, Reading FROM {table} ),
base_locations AS (SELECT * FROM base LEFT JOIN clean_locs ON base.Monitor = clean_locs.monitor ),
result AS (SELECT * FROM base_locations LEFT JOIN weather ON base_locations.Date_Time = weather.date )
SELECT 
  Chemical  AS chemical, 
  Monitor   AS monitor, 
  Date_time AS reading_stamp,
  Reading   AS reading,
  x         AS sensor_x, 
  y         AS sensor_y,
  date      AS weather_stamp, 
  wind_direction, wind_speed
FROM result WHERE date IS NOT NULL'''.format(table=table_names[2])

q_methyl = '''
WITH base AS (SELECT Chemical, Monitor, Date_Time, Reading FROM {table} ),
base_locations AS (SELECT * FROM base LEFT JOIN clean_locs ON base.Monitor = clean_locs.monitor ),
result AS (SELECT * FROM base_locations LEFT JOIN weather ON base_locations.Date_Time = weather.date )
SELECT 
  Chemical  AS chemical, 
  Monitor   AS monitor, 
  Date_time AS reading_stamp,
  Reading   AS reading,
  x         AS sensor_x, 
  y         AS sensor_y,
  date      AS weather_stamp, 
  wind_direction, wind_speed
FROM result WHERE date IS NOT NULL'''.format(table=table_names[3])


final_app = psql(q_app)
final_agoc = psql(q_agoc)
final_chloro = psql(q_chloro)
final_methyl = psql(q_methyl)

C:\Users\justi\AppData\Local\Temp\ipykernel_6308\46447345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_locs.monitor = list(range(1,10))


In [12]:
# ok i mistakenly changed the wind direction from origin degree to destination, going to change that back so vectors point
# to potential origin factory

final_app.wind_direction = final_app.wind_direction.apply(lambda x: x-180.0 if x >= 180.0 else x+180)
final_agoc.wind_direction = final_agoc.wind_direction.apply(lambda x: x-180.0 if x >= 180.0 else x+180)
final_chloro.wind_direction = final_chloro.wind_direction.apply(lambda x: x-180.0 if x >= 180.0 else x+180)
final_methyl.wind_direction = final_methyl.wind_direction.apply(lambda x: x-180.0 if x >= 180.0 else x+180)

In [19]:
# now lets join in the factories coordinates to each of these dfs

factories = locations[~locations.Name.str.contains('Sensor')]
factories.columns = 'monitor,sensor_x,sensor_y'.split(',')
factories['chemical'] = ['Factory'] * len(factories)
factories['reading_stamp'] = [np.nan] * len(factories)
factories['reading'] = [np.nan] * len(factories)
factories['weather_stamp'] = [np.nan] * len(factories)
factories['wind_direction'] = [np.nan] * len(factories)
factories['wind_speed'] = [np.nan] * len(factories)
factories = factories['chemical,monitor,reading_stamp,reading,sensor_x,sensor_y,weather_stamp,wind_direction,wind_speed'.split(',')]

final_app = pd.concat([final_app, factories])
final_agoc = pd.concat([final_agoc, factories])
final_chloro = pd.concat([final_chloro, factories])
final_methyl = pd.concat([final_methyl, factories])

C:\Users\justi\AppData\Local\Temp\ipykernel_6308\1422177362.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factories['chemical'] = ['Factory'] * len(factories)
C:\Users\justi\AppData\Local\Temp\ipykernel_6308\1422177362.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factories['reading_stamp'] = [np.nan] * len(factories)
C:\Users\justi\AppData\Local\Temp\ipykernel_6308\1422177362.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [20]:
final_app

,chemical,monitor,reading_stamp,reading,sensor_x,sensor_y,weather_stamp,wind_direction,wind_speed
0,Appluimonia,6,2016-04-29 09:00:00,8.95095,102,22,2016-04-29 09:00:00,46.3,0.2
1,Appluimonia,9,2016-12-05 12:00:00,8.51651,119,42,2016-12-05 12:00:00,174.5,1.9
2,Appluimonia,9,2016-12-18 09:00:00,7.63828,119,42,2016-12-18 09:00:00,173.6,2.2
3,Appluimonia,9,2016-12-17 09:00:00,6.37533,119,42,2016-12-17 09:00:00,164.0,1.9
0,Factory,Kasios Office Furniture,NaN,NaN,90,21,NaN,NaN,NaN
1,Factory,Radiance ColourTek,NaN,NaN,109,26,NaN,NaN,NaN
2,Factory,Indigo Sol Boards,NaN,NaN,120,22,NaN,NaN,NaN
3,Factory,Roadrunner Fitness Electronics,NaN,NaN,89,27,NaN,NaN,NaN


In [21]:
final_agoc

,chemical,monitor,reading_stamp,reading,sensor_x,sensor_y,weather_stamp,wind_direction,wind_speed
0,AGOC-3A,3,2016-08-13 09:00:00,85.31459,76,41,2016-08-13 09:00:00,131.6,1.3
1,AGOC-3A,5,2016-08-12 12:00:00,84.63599,103,43,2016-08-12 12:00:00,209.9,2.6
2,AGOC-3A,3,2016-12-05 06:00:00,80.41210,76,41,2016-12-05 06:00:00,155.2,0.8
3,AGOC-3A,6,2016-04-06 06:00:00,69.16677,102,22,2016-04-06 06:00:00,270.8,0.9
4,AGOC-3A,4,2016-12-18 09:00:00,65.71898,88,45,2016-12-18 09:00:00,173.6,2.2
0,Factory,Kasios Office Furniture,NaN,NaN,90,21,NaN,NaN,NaN
1,Factory,Radiance ColourTek,NaN,NaN,109,26,NaN,NaN,NaN
2,Factory,Indigo Sol Boards,NaN,NaN,120,22,NaN,NaN,NaN
3,Factory,Roadrunner Fitness Electronics,NaN,NaN,89,27,NaN,NaN,NaN


In [22]:
final_chloro

,chemical,monitor,reading_stamp,reading,sensor_x,sensor_y,weather_stamp,wind_direction,wind_speed
0,Chlorodinine,6,2016-04-27 00:00:00,12.84422,102,22,2016-04-27 00:00:00,306.6,1.2
1,Chlorodinine,6,2016-04-09 15:00:00,11.79647,102,22,2016-04-09 15:00:00,294.9,2.6
0,Factory,Kasios Office Furniture,NaN,NaN,90,21,NaN,NaN,NaN
1,Factory,Radiance ColourTek,NaN,NaN,109,26,NaN,NaN,NaN
2,Factory,Indigo Sol Boards,NaN,NaN,120,22,NaN,NaN,NaN
3,Factory,Roadrunner Fitness Electronics,NaN,NaN,89,27,NaN,NaN,NaN


In [23]:
final_methyl

,chemical,monitor,reading_stamp,reading,sensor_x,sensor_y,weather_stamp,wind_direction,wind_speed
0,Methylosmolene,6,2016-12-02 03:00:00,80.54764,102,22,2016-12-02 03:00:00,272.1,3.2
0,Factory,Kasios Office Furniture,NaN,NaN,90,21,NaN,NaN,NaN
1,Factory,Radiance ColourTek,NaN,NaN,109,26,NaN,NaN,NaN
2,Factory,Indigo Sol Boards,NaN,NaN,120,22,NaN,NaN,NaN
3,Factory,Roadrunner Fitness Electronics,NaN,NaN,89,27,NaN,NaN,NaN
